In [1]:
!pip install dash pandas plotly

# Import libraries
from dash import Dash, dcc, html, Input, Output, dash_table
import plotly.express as px
import pandas as pd
from google.colab import files
import io

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 41.9 MB/s eta 0:00:00


In [2]:
# Upload the file
uploaded = files.upload()
file_name = next(iter(uploaded))
df = pd.read_csv(io.BytesIO(uploaded[file_name]))

Saving cleaned_housing_data.csv to cleaned_housing_data.csv


In [3]:
app = Dash(__name__)

In [4]:
# Create dropdown options
numeric_cols = ['price', 'area', 'bedrooms', 'bathrooms', 'stories', 'price_per_sqft', 'bed_bath_ratio']
binary_cols = ['mainroad', 'guestroom', 'basement', 'hotwaterheating', 'airconditioning', 'parking', 'prefarea']
categorical_cols = ['furnishingstatus']

In [5]:
# App layout
app.layout = html.Div([
    html.H1("Real Estate Dashboard", style={'textAlign': 'center'}),

    # Filters
    html.Div([
        html.Div([
            html.Label("Price Range:"),
            dcc.RangeSlider(
                id='price-slider',
                min=df['price'].min(),
                max=df['price'].max(),
                step=100000,
                marks={i: f"{i//1000000}M" for i in range(int(df['price'].min()), int(df['price'].max())+1, 2000000)},
                value=[df['price'].min(), df['price'].max()]
            )
        ], style={'padding': '10px'}),

        html.Div([
            html.Label("Area (sqft):"),
            dcc.RangeSlider(
                id='area-slider',
                min=df['area'].min(),
                max=df['area'].max(),
                step=100,
                marks={i: str(i) for i in range(int(df['area'].min()), int(df['area'].max())+1, 2000)},
                value=[df['area'].min(), df['area'].max()]
            )
        ], style={'padding': '10px'}),

        html.Div([
            html.Label("Bedrooms:"),
            dcc.Dropdown(
                id='bedrooms-dropdown',
                options=[{'label': str(i), 'value': i} for i in sorted(df['bedrooms'].unique())],
                value=None,
                multi=True,
                placeholder="Select Bedrooms"
            )
        ], style={'width': '20%', 'padding': '10px', 'display': 'inline-block'}),

        html.Div([
            html.Label("Bathrooms:"),
            dcc.Dropdown(
                id='bathrooms-dropdown',
                options=[{'label': str(i), 'value': i} for i in sorted(df['bathrooms'].unique())],
                value=None,
                multi=True,
                placeholder="Select Bathrooms"
            )
        ], style={'width': '20%', 'padding': '10px', 'display': 'inline-block'}),

        html.Div([
            html.Label("Amenities:"),
            dcc.Checklist(
                id='amenities-checklist',
                options=[
                    {'label': ' Main Road', 'value': 'mainroad'},
                    {'label': ' Guest Room', 'value': 'guestroom'},
                    {'label': ' Basement', 'value': 'basement'},
                    {'label': ' Hot Water', 'value': 'hotwaterheating'},
                    {'label': ' AC', 'value': 'airconditioning'},
                    {'label': ' Preferred Area', 'value': 'prefarea'}
                ],
                value=[],
                inline=True
            )
        ], style={'padding': '10px'})
    ], style={'border': '1px solid #ddd', 'borderRadius': '5px', 'margin': '10px'}),

    # Charts
    html.Div([
        dcc.Graph(id='price-distribution'),
        dcc.Graph(id='area-vs-price'),
        dcc.Graph(id='amenities-correlation')
    ], style={'display': 'flex', 'flexWrap': 'wrap'}),

    # Data table
    html.Div([
        html.H3("Property Listings"),
        html.Div(id='data-table')
    ], style={'padding': '20px'})
])


In [6]:
# Callbacks
@app.callback(
    [Output('price-distribution', 'figure'),
     Output('area-vs-price', 'figure'),
     Output('amenities-correlation', 'figure'),
     Output('data-table', 'children')],
    [Input('price-slider', 'value'),
     Input('area-slider', 'value'),
     Input('bedrooms-dropdown', 'value'),
     Input('bathrooms-dropdown', 'value'),
     Input('amenities-checklist', 'value')]
)
def update_output(price_range, area_range, bedrooms, bathrooms, amenities):
    # Filter data
    filtered_df = df[
        (df['price'].between(price_range[0], price_range[1])) &
        (df['area'].between(area_range[0], area_range[1]))
    ]

    if bedrooms:
        filtered_df = filtered_df[filtered_df['bedrooms'].isin(bedrooms)]
    if bathrooms:
        filtered_df = filtered_df[filtered_df['bathrooms'].isin(bathrooms)]

    for amenity in amenities:
        filtered_df = filtered_df[filtered_df[amenity] == 1]

    # Create figures
    price_fig = px.histogram(
        filtered_df,
        x='price',
        title='Price Distribution',
        nbins=30,
        color_discrete_sequence=['#3498db']
    )

    area_fig = px.scatter(
        filtered_df,
        x='area',
        y='price',
        color='bedrooms',
        title='Area vs Price',
        hover_data=['bathrooms', 'stories'],
        trendline='lowess'
    )

    amenities_fig = px.imshow(
        filtered_df[binary_cols].corr(),
        title='Amenities Correlation',
        color_continuous_scale='RdBu',
        zmin=-1,
        zmax=1
    )

    # Create data table
    table = dash_table.DataTable(
        data=filtered_df.to_dict('records'),
        columns=[{'name': col, 'id': col} for col in filtered_df.columns],
        page_size=10,
        style_table={'overflowX': 'auto'},
        style_cell={
            'minWidth': '100px', 'width': '100px', 'maxWidth': '100px',
            'whiteSpace': 'normal',
            'textAlign': 'left'
        },
        style_header={
            'backgroundColor': '#2c3e50',
            'color': 'white',
            'fontWeight': 'bold'
        }
    )

    return price_fig, area_fig, amenities_fig, table

In [8]:
# Run the app
if __name__ == '__main__':
    from google.colab.output import eval_js
    print("Click on the URL below to access the dashboard:")
    print(eval_js("google.colab.kernel.proxyPort(8050)"))
    app.run(host='0.0.0.0', port=8050, debug=False)

Click on the URL below to access the dashboard:
https://8050-m-s-bw50w5ia8dgp-b.us-central1-0.prod.colab.dev


<IPython.core.display.Javascript object>